In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/MyDrive/subfolder/Global_local

In [ ]:
import os
import numpy as np
import pickle
import torch
from torch.optim import Adam
from torchvision import models
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import random
from torch.utils.data import DataLoader
import time
import h5py
import imutils
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
import torchvision
from torch.optim import lr_scheduler
from torch.autograd import Variable
import torch.nn as nn
import torch
from torch import autograd
import cv2
import pandas as pd
import PIL

In [ ]:
#define customized model

cls = 9

class DoubleConv(nn.Module):
    def __init__(self, in_ch, out_ch):
        super(DoubleConv, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, 3, padding=1),
            nn.BatchNorm2d(out_ch), 
            nn.ReLU(inplace=True),
            nn.Conv2d(out_ch, out_ch, 3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True)
        )
    def forward(self, input):
        return self.conv(input)

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv0 = nn.Conv2d(3, 16, 7, stride=2, padding = 4)  #224
        self.pool0 = nn.MaxPool2d(3,2)
        self.conv1 = DoubleConv(16, 32)  # 56
        self.pool1 = nn.MaxPool2d(2)
        self.conv2 = DoubleConv(32, 64) # 28
        self.pool2 = nn.MaxPool2d(2)
        # self.conv3 = DoubleConv(64, 128) # 14
        self.conv3 = nn.Conv2d(64, 128, 3, padding = 1)
        self.pool3 = nn.MaxPool2d(2)
        # self.conv4 = DoubleConv(128, 256) # 7
        self.conv4 = nn.Conv2d(128, 256, 3, padding = 1)

        # self.pool4 = nn.MaxPool3d(2)
        # self.conv5 = DoubleConv(256, 512)

        self.fc1 = nn.Linear(256*7*7, 256)
        self.fc2 = nn.Linear(256, cls)
        self.dropout = nn.Dropout(0.5)
        self.dropout0 = nn.Dropout(0.25)
        # self.dropout1 = nn.Dropout(0.5)
    def forward(self, x):
        c0 = self.conv0(x)     
        p0 = F.relu(self.pool0(c0))
        # print(p0.shape)
        c1 = self.conv1(p0)
        p1 = self.pool1(c1)
        # p1 = self.dropout0(p1)
        c2 = self.conv2(p1)
        p2 = self.pool2(c2)
        p2 = self.dropout(p2)
        c3 = self.conv3(p2)
        p3 = F.relu(self.pool3(c3))
        p3 = self.dropout(p3)
        c4 = F.relu(self.conv4(p3))
        out = c4.view(-1, 256*7*7)
        out = F.relu(self.fc1(out))
        out = self.fc2(out)
        # out = nn.LogSoftmax(dim = 1)(x)
        return out

model = CNN()
model.cuda()
model.train()

from torchsummary import summary
summary(model, input_size=(3,224,224))
criterion = nn.CrossEntropyLoss()


In [ ]:
## grating stims
# read pickle file
with open('Customized_Gratings/Small_Gratings.pkl','rb') as f:
  gratings = pickle.load(f)
G_train, G_test = train_test_split(gratings,test_size = 0.1, shuffle = True, random_state = 1)
cls = 9
times = 10

In [ ]:
## Non-random grating generator

cls = 9
times = 10
#train index
num = len(gratings)
div = 180/cls
index_tmp = np.repeat(np.arange(num),cls*cls)
k = []
for i in range(cls):
  k.extend([i*div])
k = k*num*cls
l = []
for i in range(cls):
  l.extend([i*div])
l = l*num
l = np.repeat(l,cls)
data_index = [i for i in zip(index_tmp,k,l)] 
## grating based on requirements specified in index

class myDataset():
    """
    Args:
              input input_file_list = [f_content, f_angle]
              target_file_list = f_trans      
              patch_size: int , the cubic patch parameter     
    """
    def __init__(self,input_patches,index,cls = cls, input_size=224,random = False):

        self.index = index
        self.inputs = input_patches
        self.input_size = input_size
        self.div = 180/cls
       
    def __len__(self):
        return len(self.index)

    @staticmethod
    def rotate(img,angle):
        return imutils.rotate(img, angle=angle)

    @staticmethod
    ##Randomly distributed with minimum distance
    def random_distributed_with_minimum(n_points,half_distance,boundary_length):
      position = []
      occupied_length = 0
      remain_length = boundary_length 
      def set_first_remain_point(remain_n,remain_length):
          return random.randint(half_distance, remain_length - remain_n * half_distance *2 - half_distance) + occupied_length
      for i in range(n_points,0,-1):
          remain_n = i - 1
          p = set_first_remain_point(remain_n,remain_length)
          occupied_length = p + half_distance
          remain_length = boundary_length - occupied_length
          position.append(p)
      sign = random.randint(0,1)
      if sign:
        position =[(boundary_length - i) for i in position]
      return position
    def __getitem__(self, item):
        global p,img_0,n,img0,a
        idx, angle_l, angle_g = self.index[item]  
        if item == 0 or self.index[item-1][0] != self.index[item][0]:
            img_0 = self.inputs[idx]
            #uniform with random shading
            img_0 = img_0/255*random.uniform(0.5,1)
            a = img_0.shape[1]  #half of the patch's side length
            #num of gratings & generate random distributed positions
            n = 2 if self.input_size//a == 2 else random.randint(2,self.input_size//a)
            p = self.random_distributed_with_minimum(n,np.int(a//2),self.input_size)
            #Initialize with Gaussian noise
            noise_sigma = random.uniform(0,0.3)
            img0 = np.random.randn(3,self.input_size,self.input_size) * noise_sigma
        
        #random rotate locally 
        img = self.rotate(img_0.swapaxes(0,2),angle_l-angle_g).swapaxes(0,2)

        img_1 = np.zeros((3,self.input_size,self.input_size))
        img_out = img0.copy()
        #merge based on random distributed positions
        for i in p:
          # print(int(a//2),i,'\n')
          img_1[:,int(self.input_size//2-a//2):int(self.input_size//2+a//2),i-int(a//2):i+int(a//2)] += img

        #rotate globally based on label difference
        img_1 = self.rotate(img_1.swapaxes(0,2),angle_g).swapaxes(0,2)
        img_out += img_1
        img_out =  (img_out-np.min(img_out))/(np.max(img_out)-np.min(img_out))
        label_l = np.int(angle_l//self.div)
        label_g = np.int(angle_g//self.div)

        return torch.from_numpy(img_out).float(), label_l,label_g
batch_size = 275
dataset = myDataset(gratings,data_index)

Local Orientation

In [ ]:
model_name = 'model_CNN_local_muilti_angle'
model.load_state_dict(torch.load("CNN_model/"+model_name))

In [ ]:
##save all convolutional layers
 
model_weights = [] # we will save the conv layer weights in this list
conv_layers = [] # we will save the 49 conv layers in this list
model_children = list(model.children())
# counter to keep count of the conv layers
counter = 0 
# append all the conv layers and their respective weights to the list
for i in range(len(model_children)):
    if (type(model_children[i]) == nn.Conv2d) :
        counter += 1
        model_weights.append(model_children[i].weight)
        conv_layers.append(model_children[i])
    elif type(model_children[i]) == DoubleConv:
          for child in model_children[i].conv.children():
                if type(child) == nn.Conv2d:
                    counter += 1
                    model_weights.append(child.weight)
                    conv_layers.append(child)
 
print(f"Total convolutional layers: {counter}")

In [ ]:
# take a look at the conv layers and the respective weights
for weight, conv in zip(model_weights, conv_layers):
    # print(f"WEIGHT: {weight} \nSHAPE: {weight.shape}")
    print(f"CONV: {conv} ====> SHAPE: {weight.shape}")
# visualize the first conv layer filters
plt.figure(figsize=(20, 17))
for i, filter in enumerate(model_weights[0]):
    plt.subplot(8, 8, i+1) # (8, 8) because in conv0 we have 7x7 filters and total of 64 (see printed shapes)
    plt.imshow(filter[0, :, :].cpu().detach(), cmap='gray')
    plt.axis('off')
    # plt.savefig('../outputs/filter.png')
plt.show()

In [ ]:
class SaveFeatures():
    """注册hook和移除hook
    """
    def __init__(self, module):
        self.hook = module.register_forward_hook(self.hook_fn)
    def hook_fn(self, module, input, output):
        # self.features = output.clone().detach().requires_grad_(True)
        self.features = output.clone()
    def close(self):
        self.hook.remove()

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# 生成随机的图片
sz = 224
# 超参数
lr = 0.1 # 学习率
opt_steps = 5 # 迭代次数
upscaling_steps = 1 # 图像放大次数
# blur=3
# upscaling_factor=1.2 # 把图像变粗
for layer in range(5,len(conv_layers)):
  for filter in range(10000):
# 定义处理时的均值与方差
# img = np.uint8(np.random.uniform(150, 180, (3, sz, sz)))/255
    img = np.uint8(np.random.randn(3, sz, sz))
    img = (img - np.min(img))/(np.max(img)-np.min(img))
    # img[None]可以增加一个维度
    img = torch.from_numpy(img[None]).float().to(device)
    activations = SaveFeatures(conv_layers[layer])
    # activations = SaveFeatures(list(model.children())[2].conv[4])
    for epoch in range(upscaling_steps):  # scale the image up upscaling_steps times
        # --------------------------------------------------------------------------------
        # 因为原始的网络对图片做了normalization, 所以这里对输入图片也要做normalization
        # --------------------------------------------------------------------------------
        # img = (img - cnn_normalization_mean) / cnn_normalization_std     
        # print('Imgshape1 : ',img.shape)
        img_var = Variable(img, requires_grad=True)  # convert image to Variable that requires grad
        # ----------
        # 定义优化器
        # ----------
        optimizer = torch.optim.Adam([img_var], lr=lr, weight_decay=1e-6)
        for n in range(opt_steps):  # optimize pixel values for opt_steps times
            optimizer.zero_grad()
            model(img_var) # 正向传播
            if (filter == 0) and  (n == 0):
                max_fil = activations.features.shape[1]
                print('maximum filter', max_fil)
               
            loss = -F.relu(activations.features)[0, filter].mean() # loss相当于最大该层的激活的值
            loss.backward()
            # print(loss)
            optimizer.step()
        # ------------
        # 图像进行还原
        # ------------
        # print('Loss:',loss.cpu().detach().numpy())
        # img = img_var * cnn_normalization_std + cnn_normalization_mean # 这个使用img_var变换img
 
        img = img_var.data.cpu().numpy()[0].transpose(1,2,0)
        img = (img - np.min(img))/(np.max(img)-np.min(img))
        # sz = int(upscaling_factor * sz)  # calculate new image size
        # img = cv2.resize(img, (sz, sz), interpolation = cv2.INTER_CUBIC)  # scale image up
        # if blur is not None: img = cv2.blur(img,(blur,blur))  # blur image to reduce high frequency patterns
        # print('Imgshape : ',img.shape)
        img = torch.from_numpy(img.transpose(2,0,1)[None]).to(device)
        # print('Imgshape3 : ',img.shape)
        # print(str(epoch),',Finished')
        # print('=======')
    activations.close() # 移除hook
    #最后将图像进行保存即可。
 
    # 保存图片
    image = img.cpu().clone()
    image = image.squeeze(0)
    unloader = transforms.ToPILImage()
    image = unloader(image)
    image = cv2.cvtColor(np.asarray(image),cv2.COLOR_RGB2BGR)
    # image = cv2.resize(image, (56,56), interpolation = cv2.INTER_CUBIC)
    if filter%16 == 0:
        print('=======================Conv_layer%d_part%d========================='%(layer,filter//16))
        fig = plt.figure(figsize = (32 ,8))
        ax = fig.subplots(2,8)
        plt.suptitle('Conv_layer%d_part%d'%(layer,filter//16),y = 1)
    ax[filter%16//8,filter%8].imshow(image)
    ax[filter%16//8,filter%8].axis('off')
    if filter%16 == 15:
      plt.savefig('filters/%s_opt%dsteps_Conv_layer%d_part%d'%(model_name,opt_steps,layer,filter//16), dpi=300)
      plt.tight_layout() 
      plt.show()
      plt.close()
      
    # cv2.imwrite('res1.jpg',image)
    torch.cuda.empty_cache()
    if filter+1 == max_fil:
      # print('Loss:',loss.cpu().detach().numpy())
      break

In [ ]:
##find the filters with max activation 
img = []
img1 = []
idx = 81
for i in range(cls):
  id = i*9 + idx
  data, l, g = dataset[id]
  data.unsqueeze_(0)
  data = data.cuda()
  img.append(data)
  # ax[i].imshow(data[0,].cpu().numpy().swapaxes(0,2).swapaxes(0,1))
  # ax[i].axis('off')
  if i == 2:
    img1.append(data)
    for j in range(1,cls):
      id = i*9 + idx + j
      data, l, g = dataset[id]
      data.unsqueeze_(0)
      data = data.cuda()
      img1.append(data)

input_img = img1

print('input images')
fig = plt.figure(figsize = (5*cls,5))
ax = fig.subplots(1,cls)
for i, data in enumerate(input_img):
  ax[i].imshow(data[0,].cpu().numpy().swapaxes(0,2).swapaxes(0,1))
  ax[i].axis('off')
plt.show()
plt.close()

for i in range(len(conv_layers)):
  fig = plt.figure(figsize = (8,25))
  plt.title('conv_layer{}'.format(i))
  for j, data in enumerate(input_img):
    activations = SaveFeatures(conv_layers[i])
    output = model(data)
    max_fil = activations.features[0,].cpu().detach().numpy().shape[0]
    avg_act = np.mean(F.relu(activations.features)[0,].cpu().detach().numpy().reshape(max_fil,-1),axis = 1).copy()
    activations.close() # 移除hook
    torch.cuda.empty_cache()
    plt.subplot(9,1,j+1)
    
    plt.text(0.64*max_fil,0.95*np.max(avg_act),'Max 5 activations:{}'.format(np.argsort(avg_act)[::-1][:5]))
    plt.bar(np.arange(max_fil),avg_act,0.9)
  plt.tight_layout() 
  plt.show()
  plt.close()


In [ ]:
##Find the filter with max activation in each orientation

# Filter visualization function
def filter_vis(layer,filter,conv_layers = conv_layers,opt_steps = 20,lr = 0.1,sz = 224):
  ##Function for filter visualization
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  img = np.uint8(np.random.randn(3, sz, sz))
  img = (img - np.min(img))/(np.max(img)-np.min(img))

  img = torch.from_numpy(img[None]).float().to(device)
  activations = SaveFeatures(conv_layers[layer])
  img_var = Variable(img, requires_grad=True)  # convert image to Variable that requires grad

  optimizer = torch.optim.Adam([img_var], lr=lr, weight_decay=1e-6)
  for n in range(opt_steps):  # optimize pixel values for opt_steps times
      optimizer.zero_grad()
      model(img_var) # 正向传播
      if (filter == 0) and  (n == 0):
          max_fil = activations.features.shape[1]
          print('maximum filter', max_fil)
          
      loss = -F.relu(activations.features)[0, filter].mean() # loss相当于最大该层的激活的值
      loss.backward()
      optimizer.step()

  print('Loss:',loss.cpu().detach().numpy())
  img = img_var.data.cpu().numpy()[0].transpose(1,2,0)
  img = (img - np.min(img))/(np.max(img)-np.min(img))
  img = torch.from_numpy(img.transpose(2,0,1)[None]).to(device)

  activations.close() # 移除hook
  #最后将图像进行保存即可。

  # 保存图片
  image = img.cpu().clone()
  image = image.squeeze(0)
  unloader = transforms.ToPILImage()
  image = unloader(image)
  image = cv2.cvtColor(np.asarray(image),cv2.COLOR_RGB2BGR)
  plt.imshow(image)
  plt.savefig('filters/%s_opt%dsteps_Conv_layer%d_filter%d'%(model_name,opt_steps,layer,filter), dpi=300)
  plt.show()
  plt.close()
  torch.cuda.empty_cache()

##find the filters with max activation 
df = pd.DataFrame(columns=['id','local','global'] + ['layer%d'%i for i in range(7)])
idx = 81
for i in range(cls*cls*100):
  id = i +idx
  data, l, g = dataset[id]
  data.unsqueeze_(0)
  data = data.cuda()
  max = [id,l,g]
  for j in range(len(conv_layers)):
      activations = SaveFeatures(conv_layers[j])
      output = model(data)
      max_fil = activations.features[0,].cpu().detach().numpy().shape[0]
      avg_act = np.mean(F.relu(activations.features)[0,].cpu().detach().numpy().reshape(max_fil,-1),axis = 1).copy()
      # if i == 0 and np.argmax(avg_act) != 12:
      #   plt.imshow(data[0,].cpu().numpy().swapaxes(0,2).swapaxes(0,1))
      #   plt.show()
      #   plt.close()
      max.append(np.argmax(avg_act))
      activations.close() # 移除hook
      torch.cuda.empty_cache()
  df.loc[i] = max
display(df)

for layer in range(7):
  for filter in df[df['local'] == 2]['layer%d'%layer].unique():
      print('### layer%d filter%d'%(layer,filter))
      print(df[df['layer%d'%layer] == filter]['local'].value_counts())
      print(df[df['layer%d'%layer] == filter]['global'].value_counts())
      filter_vis(layer,filter)

Global Orientation

In [ ]:
model_name = 'model_CNN_global_9angles'
model.load_state_dict(torch.load("CNN_model/"+model_name))

In [ ]:
##save all convolutional layers
 
model_weights = [] # we will save the conv layer weights in this list
conv_layers = [] # we will save the 49 conv layers in this list
model_children = list(model.children())
# counter to keep count of the conv layers
counter = 0 
# append all the conv layers and their respective weights to the list
for i in range(len(model_children)):
    if (type(model_children[i]) == nn.Conv2d) :
        counter += 1
        model_weights.append(model_children[i].weight)
        conv_layers.append(model_children[i])
    elif type(model_children[i]) == DoubleConv:
          for child in model_children[i].conv.children():
                if type(child) == nn.Conv2d:
                    counter += 1
                    model_weights.append(child.weight)
                    conv_layers.append(child)
 
print(f"Total convolutional layers: {counter}")

In [ ]:
# take a look at the conv layers and the respective weights
for weight, conv in zip(model_weights, conv_layers):
    # print(f"WEIGHT: {weight} \nSHAPE: {weight.shape}")
    print(f"CONV: {conv} ====> SHAPE: {weight.shape}")
# visualize the first conv layer filters
plt.figure(figsize=(20, 17))
for i, filter in enumerate(model_weights[0]):
    plt.subplot(8, 8, i+1) # (8, 8) because in conv0 we have 7x7 filters and total of 64 (see printed shapes)
    plt.imshow(filter[0, :, :].cpu().detach(), cmap='gray')
    plt.axis('off')
    # plt.savefig('../outputs/filter.png')
plt.show()

In [ ]:
class SaveFeatures():
    """注册hook和移除hook
    """
    def __init__(self, module):
        self.hook = module.register_forward_hook(self.hook_fn)
    def hook_fn(self, module, input, output):
        # self.features = output.clone().detach().requires_grad_(True)
        self.features = output.clone()
    def close(self):
        self.hook.remove()

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# 生成随机的图片
sz = 224
# 超参数
lr = 0.1 # 学习率
opt_steps = 25 # 迭代次数
upscaling_steps = 1 # 图像放大次数
# blur=3
# upscaling_factor=1.2 # 把图像变粗
for layer in range(len(conv_layers)):
  for filter in range(10000):
# 定义处理时的均值与方差
# img = np.uint8(np.random.uniform(150, 180, (3, sz, sz)))/255
    img = np.uint8(np.random.randn(3, sz, sz))
    img = (img - np.min(img))/(np.max(img)-np.min(img))
    # img[None]可以增加一个维度
    img = torch.from_numpy(img[None]).float().to(device)
    activations = SaveFeatures(conv_layers[layer])
    # activations = SaveFeatures(list(model.children())[2].conv[4])
    for epoch in range(upscaling_steps):  # scale the image up upscaling_steps times
        # --------------------------------------------------------------------------------
        # 因为原始的网络对图片做了normalization, 所以这里对输入图片也要做normalization
        # --------------------------------------------------------------------------------
        # img = (img - cnn_normalization_mean) / cnn_normalization_std     
        # print('Imgshape1 : ',img.shape)
        img_var = Variable(img, requires_grad=True)  # convert image to Variable that requires grad
        # ----------
        # 定义优化器
        # ----------
        optimizer = torch.optim.Adam([img_var], lr=lr, weight_decay=1e-6)
        for n in range(opt_steps):  # optimize pixel values for opt_steps times
            optimizer.zero_grad()
            model(img_var) # 正向传播
            if (filter == 0) and  (n == 0):
                max_fil = activations.features.shape[1]
                print('maximum filter', max_fil)
               
            loss = -F.relu(activations.features)[0, filter].mean() # loss相当于最大该层的激活的值
            loss.backward()
            # print(loss)
            optimizer.step()
        # ------------
        # 图像进行还原
        # ------------
        # print('Loss:',loss.cpu().detach().numpy())
        # img = img_var * cnn_normalization_std + cnn_normalization_mean # 这个使用img_var变换img
 
        img = img_var.data.cpu().numpy()[0].transpose(1,2,0)
        img = (img - np.min(img))/(np.max(img)-np.min(img))
        # sz = int(upscaling_factor * sz)  # calculate new image size
        # img = cv2.resize(img, (sz, sz), interpolation = cv2.INTER_CUBIC)  # scale image up
        # if blur is not None: img = cv2.blur(img,(blur,blur))  # blur image to reduce high frequency patterns
        # print('Imgshape : ',img.shape)
        img = torch.from_numpy(img.transpose(2,0,1)[None]).to(device)
        # print('Imgshape3 : ',img.shape)
        # print(str(epoch),',Finished')
        # print('=======')
    activations.close() # 移除hook
    #最后将图像进行保存即可。
 
    # 保存图片
    image = img.cpu().clone()
    image = image.squeeze(0)
    unloader = transforms.ToPILImage()
    image = unloader(image)
    image = cv2.cvtColor(np.asarray(image),cv2.COLOR_RGB2BGR)
    # image = cv2.resize(image, (56,56), interpolation = cv2.INTER_CUBIC)
    if filter%16 == 0:
        print('=======================Conv_layer%d_part%d========================='%(layer,filter//16))
        fig = plt.figure(figsize = (32 ,8))
        ax = fig.subplots(2,8)
        plt.suptitle('Conv_layer%d_part%d'%(layer,filter//16),y = 1)
    ax[filter%16//8,filter%8].imshow(image)
    ax[filter%16//8,filter%8].axis('off')
    if filter%16 == 15:
      plt.savefig('filters/%s_opt%dsteps_Conv_layer%d_part%d'%(model_name,opt_steps,layer,filter//16), dpi=300)
      plt.tight_layout() 
      plt.show()
      plt.close()
      
    # cv2.imwrite('res1.jpg',image)
    torch.cuda.empty_cache()
    if filter+1 == max_fil:
      # print('Loss:',loss.cpu().detach().numpy())
      break

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# 生成随机的图片
sz = 224
# 超参数
lr = 0.1 # 学习率
opt_steps = 50 # 迭代次数
upscaling_steps = 1 # 图像放大次数
# blur=3
# upscaling_factor=1.2 # 把图像变粗
for layer in range(len(conv_layers)):
  for filter in range(10000):
# 定义处理时的均值与方差
# img = np.uint8(np.random.uniform(150, 180, (3, sz, sz)))/255
    img = np.uint8(np.random.randn(3, sz, sz))
    img = (img - np.min(img))/(np.max(img)-np.min(img))
    # img[None]可以增加一个维度
    img = torch.from_numpy(img[None]).float().to(device)
    activations = SaveFeatures(conv_layers[layer])
    # activations = SaveFeatures(list(model.children())[2].conv[4])
    for epoch in range(upscaling_steps):  # scale the image up upscaling_steps times
        # --------------------------------------------------------------------------------
        # 因为原始的网络对图片做了normalization, 所以这里对输入图片也要做normalization
        # --------------------------------------------------------------------------------
        # img = (img - cnn_normalization_mean) / cnn_normalization_std     
        # print('Imgshape1 : ',img.shape)
        img_var = Variable(img, requires_grad=True)  # convert image to Variable that requires grad
        # ----------
        # 定义优化器
        # ----------
        optimizer = torch.optim.Adam([img_var], lr=lr, weight_decay=1e-6)
        for n in range(opt_steps):  # optimize pixel values for opt_steps times
            optimizer.zero_grad()
            model(img_var) # 正向传播
            if (filter == 0) and  (n == 0):
                max_fil = activations.features.shape[1]
                print('maximum filter', max_fil)
               
            loss = -F.relu(activations.features)[0, filter].mean() # loss相当于最大该层的激活的值
            loss.backward()
            # print(loss)
            optimizer.step()
        # ------------
        # 图像进行还原
        # ------------
        # print('Loss:',loss.cpu().detach().numpy())
        # img = img_var * cnn_normalization_std + cnn_normalization_mean # 这个使用img_var变换img
 
        img = img_var.data.cpu().numpy()[0].transpose(1,2,0)
        img = (img - np.min(img))/(np.max(img)-np.min(img))
        # sz = int(upscaling_factor * sz)  # calculate new image size
        # img = cv2.resize(img, (sz, sz), interpolation = cv2.INTER_CUBIC)  # scale image up
        # if blur is not None: img = cv2.blur(img,(blur,blur))  # blur image to reduce high frequency patterns
        # print('Imgshape : ',img.shape)
        img = torch.from_numpy(img.transpose(2,0,1)[None]).to(device)
        # print('Imgshape3 : ',img.shape)
        # print(str(epoch),',Finished')
        # print('=======')
    activations.close() # 移除hook
    #最后将图像进行保存即可。
 
    # 保存图片
    image = img.cpu().clone()
    image = image.squeeze(0)
    unloader = transforms.ToPILImage()
    image = unloader(image)
    image = cv2.cvtColor(np.asarray(image),cv2.COLOR_RGB2BGR)
    # image = cv2.resize(image, (56,56), interpolation = cv2.INTER_CUBIC)
    if filter%16 == 0:
        print('=======================Conv_layer%d_part%d========================='%(layer,filter//16))
        fig = plt.figure(figsize = (32 ,8))
        ax = fig.subplots(2,8)
        plt.suptitle('Conv_layer%d_part%d'%(layer,filter//16),y = 1)
    ax[filter%16//8,filter%8].imshow(image)
    ax[filter%16//8,filter%8].axis('off')
    if filter%16 == 15:
      plt.savefig('filters/%s_opt%dsteps_Conv_layer%d_part%d'%(model_name,opt_steps,layer,filter//16), dpi=300)
      plt.tight_layout() 
      plt.show()
      plt.close()
      
    # cv2.imwrite('res1.jpg',image)
    torch.cuda.empty_cache()
    if filter+1 == max_fil:
      # print('Loss:',loss.cpu().detach().numpy())
      break

In [ ]:
##find the filters with max activation 
img = []
img1 = []
idx = 810
for i in range(cls):
  id = i*9 + idx
  data, l, g = dataset[id]
  data.unsqueeze_(0)
  data = data.cuda()
  img.append(data)
  # ax[i].imshow(data[0,].cpu().numpy().swapaxes(0,2).swapaxes(0,1))
  # ax[i].axis('off')
  if i == 2:
    img1.append(data)
    for j in range(1,cls):
      id = i*9 + idx + j
      data, l, g = dataset[id]
      data.unsqueeze_(0)
      data = data.cuda()
      img1.append(data)

input_img = img

print('input images')
fig = plt.figure(figsize = (5*cls,5))
ax = fig.subplots(1,cls)
for i, data in enumerate(input_img):
  ax[i].imshow(data[0,].cpu().numpy().swapaxes(0,2).swapaxes(0,1))
  ax[i].axis('off')
plt.show()
plt.close()

for i in range(len(conv_layers)):
  fig = plt.figure(figsize = (8,25))
  plt.title('conv_layer{}'.format(i))
  for j, data in enumerate(input_img):
    activations = SaveFeatures(conv_layers[i])
    output = model(data)
    max_fil = activations.features[0,].cpu().detach().numpy().shape[0]
    avg_act = np.mean(F.relu(activations.features)[0,].cpu().detach().numpy().reshape(max_fil,-1),axis = 1).copy()
    activations.close() # 移除hook
    torch.cuda.empty_cache()
    plt.subplot(9,1,j+1)
    
    plt.text(0.64*max_fil,0.95*np.max(avg_act),'Max 5 activations:{}'.format(np.argsort(avg_act)[::-1][:5]))
    plt.bar(np.arange(max_fil),avg_act,0.9)
  plt.tight_layout() 
  plt.show()
  plt.close()


In [ ]:
##Find the filter with max activation in each orientation

# Filter visualization function
def filter_vis(layer,filter,conv_layers = conv_layers,opt_steps = 30,lr = 0.1,sz = 224):
  ##Function for filter visualization
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  img = np.uint8(np.random.randn(3, sz, sz))
  img = (img - np.min(img))/(np.max(img)-np.min(img))

  img = torch.from_numpy(img[None]).float().to(device)
  activations = SaveFeatures(conv_layers[layer])
  img_var = Variable(img, requires_grad=True)  # convert image to Variable that requires grad

  optimizer = torch.optim.Adam([img_var], lr=lr, weight_decay=1e-6)
  for n in range(opt_steps):  # optimize pixel values for opt_steps times
      optimizer.zero_grad()
      model(img_var) # 正向传播
      if (filter == 0) and  (n == 0):
          max_fil = activations.features.shape[1]
          print('maximum filter', max_fil)
          
      loss = -F.relu(activations.features)[0, filter].mean() # loss相当于最大该层的激活的值
      loss.backward()
      optimizer.step()

  print('Loss:',loss.cpu().detach().numpy())
  img = img_var.data.cpu().numpy()[0].transpose(1,2,0)
  img = (img - np.min(img))/(np.max(img)-np.min(img))
  img = torch.from_numpy(img.transpose(2,0,1)[None]).to(device)

  activations.close() # 移除hook
  #最后将图像进行保存即可。

  # 保存图片
  image = img.cpu().clone()
  image = image.squeeze(0)
  unloader = transforms.ToPILImage()
  image = unloader(image)
  image = cv2.cvtColor(np.asarray(image),cv2.COLOR_RGB2BGR)
  plt.imshow(image)
  plt.savefig('filters/%s_opt%dsteps_Conv_layer%d_filter%d'%(model_name,opt_steps,layer,filter), dpi=300)
  plt.show()
  plt.close()
  torch.cuda.empty_cache()

##find the filters with max activation 
df = pd.DataFrame(columns=['id','local','global'] + ['layer%d'%i for i in range(7)])
idx = 81
for i in range(cls*cls*100):
  id = i +idx
  data, l, g = dataset[id]
  data.unsqueeze_(0)
  data = data.cuda()
  max = [id,l,g]
  for j in range(len(conv_layers)):
      activations = SaveFeatures(conv_layers[j])
      output = model(data)
      max_fil = activations.features[0,].cpu().detach().numpy().shape[0]
      avg_act = np.mean(F.relu(activations.features)[0,].cpu().detach().numpy().reshape(max_fil,-1),axis = 1).copy()
      # if i == 0 and np.argmax(avg_act) != 12:
      #   plt.imshow(data[0,].cpu().numpy().swapaxes(0,2).swapaxes(0,1))
      #   plt.show()
      #   plt.close()
      max.append(np.argmax(avg_act))
      activations.close() # 移除hook
      torch.cuda.empty_cache()
  df.loc[i] = max
display(df)

for layer in range(7):
  for filter in df[df['local'] == 2]['layer%d'%layer].unique():
      print('### layer%d filter%d'%(layer,filter))
      print(df[df['layer%d'%layer] == filter]['local'].value_counts())
      print(df[df['layer%d'%layer] == filter]['global'].value_counts())
      filter_vis(layer,filter)